In [21]:
# Importing libraries
import numpy as np
from qiskit.opflow import Zero,One,X,Y,Z,I
from qiskit.circuit import Parameter, ParameterVector
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.utils import QuantumInstance
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA
from qiskit.primitives import Estimator
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# constants
hw = 7
V0 = -5.68658111
# n = Size of the basis

In [3]:
# Getting the second quantized hamiltonian for pionless EFT potential
def T(nd,ld,l,n):
    e = 0
    if ld==l:
        if n==nd:
            e = (2*n) + l + 1.5
        elif n==nd+1:
            e = -1 * np.sqrt(n*(n+l+0.5))
        elif n==nd-1:
            e = -1 * np.sqrt((n+1)*(n+l+1.5))
    return e*(hw/2)

def V(nd,n):
    e = 0
    if n==nd and n==0:
        e = V0
    return e

def Hn(n):
    term = []
    for i in range(n):
        for j in range(n):
            k = T(i,0,0,j) + V(i,j)
            if k != 0:
                term.append([i,j,k])
    return term

In [4]:
# Jordan-Wigner transformation:
def JW(n):
    H = Hn(n)
    def term(i,j,n):
        if j==n-1:
            a = (0.5*(X + 1j * Y))
            for k in range(n-1):
                a = a^Z
                
        else:
            a = I
            for k in range(n-(j+2)):
                a = a^I
            a = a^(0.5*(X + 1j * Y))
            for k in range(j):
                a = a^Z
                
        if i==n-1:
            adag = (0.5*(X + -1j * Y))
            for k in range(n-1):
                adag = adag^Z
                
        else:
            adag = I
            for k in range(n-(i+2)):
                adag = adag^I
            adag = adag^(0.5*(X + -1j * Y))
            for k in range(i):
                adag = adag^Z

        return adag@a
    
    k = 0
    for i in H:
        k = k + float(i[2])*term(i[0],i[1],n)
    return k.reduce()

In [5]:
# Function that changes an Occupation basis state to Parity basis state
def OccToParity(x):
    k = np.matmul(np.triu(np.ones(len(x))),np.transpose(x))
    m = []
    for i in range(len(k)):
        if k[i]%2 == 0:
            m.append(0)
        else:
            m.append(1)
    return m

# Function that changes an Occupation basis state to Barvyi-Kitaev basis state
def OccToBK(x):

    n = int(np.log2(len(x)))
    k = np.matmul(beta(n),np.transpose(x))
    m = []
    for i in range(len(k)):
        if k[i]%2 == 0:
            m.append(0)
        else:
            m.append(1)
            
    return m

In [6]:
# Bravyi-Kitaev transformation:
def BK(k):
    H = Hn(k)
    n = int(np.ceil(np.log2(k)))
    
    def beta(n):
        beta = [[1,1],[0,1]]
        for i in range(n-1):
            beta = np.kron(np.eye(2),beta)
            for j in range(len(beta)):
                beta[0][j] = 1
        return beta

    def parity_matrix(n):
        mat = [[1,1],[0,1]]
        for i in range(n-1):
            mat = np.kron(np.eye(2),mat)
            for j in range(int(len(mat)/2)):
                mat[j][int(len(mat)/2)] = 1
        return mat
    
    def update(j,n):
        k = []
        mat = beta(n)
        for i in range(len(mat)):
            if mat[i][(2**n)-(j+1)] == 1:
                k.append((2**n)-(i+1))
        k.remove(j)
        return k
    
    def flip(j,n):
        k = []
        mat = np.absolute(np.linalg.inv(beta(n)))
        for i in range(len(mat)):
            if mat[(2**n-(j+1))][i] == 1:
                k.append((2**n)-(i+1))
        k.remove(j)
        return k
    
    def parity(j,n):
        k = []
        mat = parity_matrix(n)
        for i in range(len(mat)):
            if mat[(2**n)-(j+1)][i] == 1:
                k.append((2**n)-(i+1))
        k.remove(j)
        return k
    
    def a(j,k,mode):
        n = int(np.ceil(np.log2(k)))
        m = []
        if j%2 == 0:
            for i in range(2**n):
                if np.isin(i,update(j,n)):
                    m.append(X)
                elif np.isin(i,parity(j,n)):
                    m.append(Z)
                elif i == j:
                    m.append((X + ((-1)**mode)*1j*Y))
                else:
                    m.append(I)
            
            op = m[k-1]
            for i in range(k-1):
                op = op^m[k-(i+2)]
    
        remainder = list(set(parity(j,n))-set(flip(j,n)))
        l = []
        if j%2 == 1:
            for i in range(2**n):
                if np.isin(i,update(j,n)):
                    m.append(X)
                    l.append(X)
                elif i == j:
                    m.append(X)
                    l.append(((-1)**mode)*1j*Y)
                elif np.isin(i,remainder):
                    m.append(Z)
                    l.append(Z)
                elif np.isin(i,list(set(parity(j,n)) - set(remainder))):
                    m.append(Z)
                    l.append(I)
                else:
                    m.append(I)
                    l.append(I)
            
            op1 = m[k-1]
            op2 = l[k-1]
            for i in range(k-1):
                op1 = op1^m[k-(i+2)]
                op2 = op2^l[k-(i+2)]
            op = op1+op2
        return 0.5*op
    
    m = 0
    for i in H:
        m = m + float(i[2])*(a(i[0],k,1)@a(i[1],k,0))
    return m.reduce()

In [7]:
def GC(n):
    
    H = Hn(n)
    
    def gray(n):
        bit = ['0','1']
        for i in range(n-1):
            bit_rev = [bit[len(bit)-(i+1)] for i in range(len(bit))]
            bit = ['0'+bit[i] for i in range(len(bit))]
            bit_rev = ['1'+bit_rev[i] for i in range(len(bit))]
            bit = bit + bit_rev
        return bit
    
    def TS_set(i,j,n):
        g = gray(n)
        T = []
        S = []
        for k in range(n):
            if g[i][n-(k+1)] == g[j][n-(k+1)]:
                T.append(k)
            else:
                S.append(k)
        return T,S

    def a(i,j,n):
        g = gray(n)
        T,S = TS_set(i,j,n)
        P = [0.5*(I + Z), 0.5*(I - Z)]
        Q = [0.5*(X + 1j*Y), 0.5*(X - 1j*Y)]
        if i==j:
            k = P[int(g[i][n-1])]
            for m in range(1,n):
                k = k^P[int(g[i][n-(m+1)])]
    
        else:
            term = [[],[]]
            for m in range(n):
                if m in T:
                    term[0].append(P[int(g[i][n-(m+1)])])
                    term[1].append(P[int(g[i][n-(m+1)])])
                elif m in S:
                    term[0].append(Q[int(g[i][n-(m+1)])])
                    term[1].append(Q[int(g[j][n-(m+1)])])
                
            p1 = term[0][0]
            p2 = term[1][0]
            for m in range(1,n):
                p1 = p1^term[0][m]
                p2 = p2^term[1][m]
            k = p1@p2
        return k.reduce()
    
    num = []
    for i in range(n):
        for j in range(i+1):
            num.append([j,i])

    k = []
    for i in range(len(H)):
        if [H[i][0],H[i][1]] in num:
            k.append(i)
    
    m = 0
    for i in k:
        m = m + float(H[i][2])*a(H[i][0],H[i][1],n)
    
    return m

In [8]:
def JW_circuit(n):
    qc = QuantumCircuit(n)
    params = ParameterVector('theta',n-1)
    qc.x(n-1)
    qc.ry(params[0],n-2)
    for i in range(1,n-1):
        qc.cx(n-(i+1),n-(i))
        qc.cry(params[i],n-(i+1),n-(i+2))
    qc.cx(0,1)
    return qc

def BK_circuit(n):
    qc = QuantumCircuit(n)
    params = ParameterVector('theta',n-1)
    qc.x(n-1)
    qc.ry(params[0],n-2)
    for i in range(1,n-1):
        qc.cx(n-(i+1),n-(i))
        qc.cry(params[i],n-(i+1),n-(i+2))
    return qc

def ansatz(n,encode):
    p1 = Parameter('theta 1')
    p2 = Parameter('theta 2')
    p3 = Parameter('theta 3')
    
    if encode=="GC":
        if n==2:
            qc = QuantumCircuit(1)
            qc.ry(p1,0)
        if n==3:
            qc = QuantumCircuit(2)
            qc.ry(p1,0)
            qc.ry(-p2,1)
            qc.cx(0,1)
            qc.ry(p2,1)
        if n==4:
            qc = QuantumCircuit(2)
            qc.ry(p2,0)
            qc.ry(p1,1)
            qc.cx(0,1)
            qc.ry(p3,1)
            
    if encode=="JW":
        qc = JW_circuit(n)
    
    if encode=="BK":
        qc = BK_circuit(n)
            
    return qc

In [9]:
# Hamiltonians for EFT Potential
EFT = { 'JW' : { 2 : JW(2), 3 : JW(3)  },
        'BK' : { 2 : BK(2), 3 : BK(3)  },
        'GC' : { 2 : 5.9067091*I - 6.34329*Z - 4.28661*X,
                 3 : 7.765855*(I^I) - 7.984145*(I^Z) - 1.859145*(Z^I) + 1.640855*(Z^Z) - 2.143305*((I^X)+(Z^X)) - 3.91312*((X^I)-(X^Z))}}

In [10]:
# Hamiltonians for Central Potential 

CP = { 'JW' : { 2 : 7.858535*(I^I) + 0.00257*(I^Z) - 7.861105*(Z^I) - 0.37778*((X^X)+(Y^Y)),
                3 : (7.858535*(I^I^I) + 0.00257*(I^I^Z) - 7.861105*(I^Z^I) - 0.37778*((I^X^X)+(I^Y^Y)) + 15.92676*((I^I^I)-(Z^I^I)) - 3.6989*((X^Z^X)+(Y^Z^Y)) + 4.123715*((X^X^I) + (Y^Y^I))).reduce()},
       'GC' : { 2 : (7.858535*I - 7.863675*Z - 0.75556*X),
                3 : (11.892645*(I^I) - 11.895215*(I^Z) - 4.03411*(Z^I) + 4.03154*(Z^Z) - 3.6989*((X^X)-(Y^Y)) + 4.123715*((X^I) - (X^Z)) - 0.37778*((I^X)+(Z^X))) },
       'BK' : { 2 : 7.858535*(I^I) + 0.00257*(I^Z) - 7.861105*(Z^Z) - 0.37778*((I^X)-(Z^X)),
                3 : (7.858535*(I^I^I) + 0.00257*(I^I^Z) - 7.861105*(I^Z^Z) - 0.37778*((I^I^X)-(I^Z^X)) + 15.92676*((I^I^I)-(Z^I^I)) - 3.6989*((Y^Y^X)-(X^Y^Y)) + 4.123715*((X^X^I)+(Y^Y^Z))).reduce() }}

In [11]:
# Hamiltonians for Reid68 Potentials

R68 = { 'JW' : 116.5294*(I^I) - 78.85401*(I^Z) - 37.67542*(Z^I) - 12.88914*((X^X)+(Y^Y)),
        'BK' : 116.5294*(I^I) - 78.85401*(I^Z) - 37.67542*(Z^Z) - 12.88914*((I^X)-(Z^X)),
        'GC' : 116.5294*I + 41.17867*Z - 25.77827*X }

In [35]:
def vqe(encode,n,H):
    circuit = ansatz(n,encode)
    vqe = VQE(estimator=Estimator(), ansatz=circuit, optimizer=SPSA(maxiter=100))
    result = vqe.compute_minimum_eigenvalue(H)
    return result.optimal_value

In [32]:
x, y = vqe('BK',2,EFT['BK'][2])

In [37]:
# EFT results
k = [[],[]]
for i in ['JW','BK','GC']:
    for j in [2,3]:
        k[j-2].append(vqe(i,j,EFT[i][j]))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC']
results.index=[2,3]
print('Effective field theory Potential')
results

Effective field theory Potential


,JW,BK,GC
2,-1.749160,-1.749160,-1.749161
3,-2.045599,-2.022094,-2.044584


In [38]:
# CP results
k = [[],[]]
for i in ['JW','BK','GC']:
    for j in [2,3]:
        k[j-2].append(vqe(i,j,CP[i][j]))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC']
results.index=[2,3]
print('Central Potential')
results

Central Potential


,JW,BK,GC
2,-0.041355,-0.041355,-0.041355
3,-1.708598,-1.708920,-1.691379


In [40]:
# R68 results
k = [[]]
for i in ['JW','BK','GC']:
        k[0].append(vqe(i,2,R68[i][2]))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC']
results.index=[2]
print('Reid68 Potential')
results

Reid68 Potential


,JW,BK,GC
2,-37.675419,-37.67542,-25.77827
